#Import cityscapes

In [ ]:
! mkdir /content/cityScapes

In [ ]:
! wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=d.a.vandendoel@students.uu.nl&password=Derkojo95!&submit=Login' https://www.cityscapes-dataset.com/login/
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1 -P /content/cityScapes
! wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3 -P /content/cityScapes

In [ ]:
! unzip /content/cityScapes/gtFine_trainvaltest.zip -d /content/cityScapes/annotations
! unzip /content/cityScapes/leftImg8bit_trainvaltest.zip -d /content/cityScapes/img

#Mount drive and import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow_addons

In [ ]:
!python -m pip install cityscapesscripts

In [ ]:
import cv2
import datetime
import itertools
import os
import PIL
import scipy
import sys
import time
import json

import numpy as np 
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.layers import SpectralNormalization
import matplotlib.pyplot as plt
from skimage import transform
from tqdm import tqdm
from random import randint, seed
from google.colab.patches import cv2_imshow
from imageio import imread
from tensorflow.keras import layers
from glob import glob
from IPython import display
from random import randint
from skimage.metrics import structural_similarity as SSIM

print(os.listdir("../content/"))

# Load image and apply mask

In [ ]:
IM_HEIGHT = 256
IM_WIDTH = 512
BUFFER_SIZE = 2975
BATCH_SIZE = 1
OUTPUT_CHANNELS = 3
MASK_HEIGTH = 64
MASK_WIDTH = 32

In [ ]:
def load(image_file, channels, semantics = False,):
  image = tf.io.read_file(image_file)
  image = tf.io.decode_png(image, channels = channels)
  image = tf.cast(image, tf.float32)

  return image

In [ ]:
def resize(input_image, height, width):
    image = tf.image.resize(
        input_image, 
        [height, width],
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
    )
  
    return image

In [ ]:
def normalize(image):
  image = image / 127.5 - 1
  return image

In [ ]:
def image_mask(image, mask_height, mask_width, num_masks, rand_seed = None):
  # random_cutout function needs mask_height * mask_width to be divisible by 2
  if (mask_height * mask_width)%2 != 0:
    raise Exception("Error! mask size must be divisible by 2")
  
  im_height = tf.cast(IM_HEIGHT, dtype=tf.dtypes.int32)
  im_width = tf.cast(IM_WIDTH, dtype=tf.dtypes.int32)  
  mask_height = tf.cast(mask_height, dtype=tf.dtypes.int32)
  mask_width = tf.cast(mask_width, dtype=tf.dtypes.int32)
  
  min_height = tf.cast(tf.math.round((mask_height/2)+15), tf.dtypes.int32)
  min_width = tf.cast(tf.math.round((mask_width/2)+10), tf.dtypes.int32)
  max_height = tf.cast(tf.math.round(im_height/10), tf.dtypes.int32)

  try:
    image_shape = image.shape
  except:
    image_shape = image.get_shape()

  if len(image_shape) == 3:
    image = tf.expand_dims(image, 0)
  elif len(image_shape) == 4:
    pass
  else:
    raise Exception('Error! Tensor shape must either be: (batch_size, image height, image width, color channels) or (image height, image width, color channels)')
  
  mask = tf.zeros_like(image, tf.dtypes.float32)[:,:,:,0:1]
  locations = []

  #create masks
  for i in range(num_masks):
    mask_height_random = tf.random.uniform(shape = (), minval= mask_height-5, maxval=mask_height+15, dtype=tf.dtypes.int32, seed=rand_seed)
    mask_height_random = mask_height_random if mask_height_random%2 == 0 else mask_height_random-1
    mask_width_random = tf.random.uniform(shape = (), minval= mask_width, maxval=mask_width+10, dtype=tf.dtypes.int32, seed=rand_seed)
    mask_width_random = mask_width_random if mask_width_random%2 == 0 else mask_width_random-1
    
    height_offset = tf.random.uniform(shape = (), minval= min_height, maxval=im_height-max_height, dtype=tf.dtypes.int32, seed=rand_seed)
    width_offset = tf.random.uniform(shape = (), minval= min_width, maxval= im_width-min_width, dtype=tf.dtypes.int32, seed=rand_seed)
    mask = tfa.image.cutout(mask, (mask_height_random, mask_width_random), (height_offset, width_offset), constant_values = 1)
    locations.append((mask_height_random, mask_width_random, height_offset, width_offset))
  
  masked_image = (1-mask) * image

    #mask = tf.zeros([image_heigt, image_width])
    #mask = mask[height_offset:height_offset+mask_height].assign(tf.ones([mask_height, mask_width]))
    #image = mask * image


  return masked_image[0], mask[0], locations

In [ ]:
def load_images(image_file):
  image = load(image_file, 3)
  image = resize(image, IM_HEIGHT, IM_WIDTH)
  masked_image, mask, locations = image_mask(image, 64, 32, 4)
  image = normalize(image)
  masked_image = normalize(masked_image)

  #semantics = tf.io.read_file(semantics_file)
  #semantics = tf.io.decode_png(semantics, channels = 0)
  #semantics = tf.cast(semantics, tf.int32)
  #semantics = resize(semantics, 256, 512)



  #masked_image, image, mask, image_file, locations
  return masked_image, image, mask, image_file, locations

# Data loader pipeline

In [ ]:
# Fill pandas dataframe with the paths to the files
def folder_to_pd(image_dir, semantics_dir):
  image_paths = {"test": [], "train": [], "val": []}
  semantics_paths = {"test": [], "train": [], "val": []}
  json_paths = {"test": [], "train": [], "val": []}

  for data_type in ["test", "train", "val"]:
    image_subdir = os.path.join(image_dir, data_type)
    for root, subdir, files in os.walk(image_subdir):
      subdir.sort()
      files.sort()
      if files:
        for file in files:
          image_paths[data_type].append(os.path.join(root, file))

  
  for data_type in ["test", "train", "val"]:
    semantics_subdir = os.path.join(semantics_dir, data_type)
    for root, subdir, files in os.walk(semantics_subdir):
      subdir.sort()
      files.sort()
      if files:
        for i,file in enumerate(files):
          if file[-9:] == "color.png":
            semantics_paths[data_type].append(os.path.join(root, file))
          if file[-12:] == "labelIds.png":
            json_paths[data_type].append(os.path.join(root, file))


  test_ds = pd.DataFrame(list(zip(image_paths["test"], semantics_paths["test"], json_paths["test"])), columns =['Image', 'Semantics', 'label'])
  train_ds = pd.DataFrame(list(zip(image_paths["train"], semantics_paths["train"], json_paths["train"])), columns =['Image', 'Semantics', 'label'])
  val_ds = pd.DataFrame(list(zip(image_paths["val"], semantics_paths["val"], json_paths["val"])), columns =['Image', 'Semantics', 'label'])

  return test_ds, train_ds, val_ds

test_pd, train_pd, val_pd = folder_to_pd("/content/cityScapes/img/leftImg8bit", "/content/cityScapes/annotations/gtFine")


test_ds = tf.data.Dataset.from_tensor_slices((test_pd["Image"], test_pd["Semantics"], test_pd["label"]))
train_ds = tf.data.Dataset.from_tensor_slices((train_pd["Image"], train_pd["Semantics"], train_pd["label"]))
val_ds = tf.data.Dataset.from_tensor_slices((val_pd["Image"], val_pd["Semantics"], val_pd["label"]))

In [ ]:
tf_train = train_ds.shuffle(train_ds.cardinality(), reshuffle_each_iteration=True)
#tf_train = tf_train.interleave(tf.data.TFRecordDataset, num_parallel_calls = tf.data.AUTOTUNE)
tf_train = tf_train.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
tf_train = tf_train.batch(BATCH_SIZE)
tf_train = tf_train.cache("/content/temporary.tfcache")
tf_train = tf_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
tf_val = val_ds.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
tf_val = tf_val.batch(BATCH_SIZE)
tf_val = tf_val.prefetch(tf.data.AUTOTUNE)

# GAN

In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
        result.add(tf.keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)))

    result.add(tf.keras.layers.LeakyReLU())

    return result

In [ ]:
def upsample(filters, size, h, w, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()

    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())
    result.add(tf.keras.layers.Lambda(lambda x: tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)))

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [ ]:
def Generator():
        down_stack = [
            downsample(64, 4, apply_batchnorm=False),  # (bs, 128, 256, 64)
            downsample(128, 4),  # (bs, 64, 128, 128)
            downsample(256, 4),  # (bs, 32, 64, 256)
            downsample(512, 4),  # (bs, 16, 32, 512)
            downsample(512, 4),  # (bs, 8, 16, 512)
            downsample(512, 4),  # (bs, 4, 8, 512)
            downsample(512, 4),  # (bs, 2, 4, 512)
            downsample(512, 4),  # (bs, 1, 2, 512)
        ]

        up_stack = [
            upsample(512, 4, 1, 2, apply_dropout=True),  # (bs, 2, 4, 1024)
            upsample(512, 4, 2, 4, apply_dropout=True),  # (bs, 4, 8, 1024)
            upsample(512, 4, 4, 8, apply_dropout=True),  # (bs, 8, 16, 1024)
            upsample(512, 4, 8, 16),  # (bs, 16, 32, 1024)
            upsample(256, 4, 16, 32),  # (bs, 32, 64, 512)
            upsample(128, 4, 32, 64),  # (bs, 64, 128, 256)
            upsample(64, 4, 64, 128),  # (bs, 128, 256, 128)
        ]

        initializer = tf.random_normal_initializer(0., 0.02)
        last_upsample = upsample(64, 4, 128, 256)
        last = tf.keras.layers.Conv2D(OUTPUT_CHANNELS, 4, strides=1, padding='same', kernel_initializer=initializer, activation='tanh')  # (bs, 256, 512, 3)

        concat = tf.keras.layers.Concatenate()
        inputs = tf.keras.layers.Input(shape=[256, 512, 3])

        x = inputs

        # Downsampling through the model
        skips = []
        for down in down_stack:
            x = down(x)
            skips.append(x)

        skips = reversed(skips[:-1])

        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            x = concat([x, skip])

        x = last_upsample(x)
        x = last(x)

        return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape=[256, 512, 3], name='input_image')
    tar = tf.keras.layers.Input(shape=[256, 512, 3], name='target_image')

    x = tf.keras.layers.concatenate([inp, tar])  # (bs, 256, 512, channels*2)

    down1 = downsample(64, 4, False)(x)  # (bs, 128, 256, 64)
    down2 = downsample(128, 4)(down1)  # (bs, 64, 128, 128)
    down3 = downsample(256, 4)(down2)  # (bs, 32, 64, 256)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 66, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1)  # (bs, 31, 63, 512)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 65, 512)

    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                  kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 62, 1)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
generator = Generator()

In [ ]:
discriminator = Discriminator()

#GAN parameters

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
LAMBDA = 100

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

def generator_loss( disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    # mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

    total_gen_loss = gan_loss + (LAMBDA * l1_loss)

    return total_gen_loss, gan_loss, l1_loss

In [ ]:
lr = 2e-4
generator_optimizer = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(lr, beta_1=0.9)

#Training

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/Scriptie/ckpt/P2P/TransposeConv'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def generate_images(model, test_input, tar, mask):  
  prediction = model(test_input, training=True)
  prediction = prediction*mask+tar*(1-mask)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

##Steps

In [ ]:
@tf.function
def train_step(input_image, mask, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator([input_image], training=True) #generate image based on input
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminator([input_image, target], training=True) #calculate discriminator loss for ground truth images
    disc_generated_output = discriminator([input_image, gen_output], training=True) #calculate discriminator loss for predicted images

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))


In [ ]:
log_dir="/content/drive/MyDrive/Scriptie/logs/im_generator/P2P/Upsample/steps/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
def fit(train_ds, test_ds, steps):
  example_input, example_target, mask_test, path, locations = next(iter(test_ds.take(1)))
  #example_input, example_target, mask_test = next(iter(test_ds.take(1)))
  start = time.time()

  for step, (input_image, target, mask, path, locations) in train_ds.repeat().take(steps).enumerate():
    if (step) % 1000 == 0:
      display.clear_output(wait=True)

      if step != 0:
        print(f'Time taken for 1000 steps: {time.time()-start:.2f} sec\n')

      start = time.time()

      generate_images(generator, example_input, example_target, mask_test)
      print(f"Step: {step//1000}k")
    
    
    #ones_x = tf.ones_like(input_image)[:, :, :, 0:1]
    #input_image = tf.concat([input_image, ones_x, ones_x*mask], axis=3)
    
    train_step(input_image, mask, target, step)

    # Training step
    if (step+1) % 10 == 0:
      print('.', end='', flush=True)


    # Save (checkpoint) the model every 5k steps
    #if (step + 1) % 5000 == 0:
    #  checkpoint.save(file_prefix=checkpoint_prefix)

##Epoch

In [ ]:
@tf.function
def train_epoch(input_image, mask, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss
    
  
def getValAccLoss(input_image, mask, target): 
  gen_output = generator(input_image, training=True)
  gen_output = gen_output*mask+ target*(1-mask) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminator([input_image, target], training=True)
  disc_generated_output = discriminator([input_image, gen_output], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/Experiment'
summary_writer_train = tf.summary.create_file_writer(log_dir + "/fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_TrainFT")
summary_writer_val = tf.summary.create_file_writer(log_dir + "/fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_ValFT")

In [ ]:
def fit(train_ds, test_ds, epochs):
  example_input, example_target, mask_test, path, locations = next(iter(test_ds.take(1)))
  example_input, example_target, mask_test
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    generate_images(generator, example_input, example_target, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(input_image, target, mask, path, locations) in train_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epoch(input_image, mask, target)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)

    
    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(input_image, target, mask, path, locations) in test_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLoss(input_image, mask, target)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 5 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)


#Train

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir}

In [ ]:
fit(tf_train, tf_val, 20)

# Metrics

In [ ]:
def metrics(image_1, image_2):
  # calculate L1 distance
  L1_distance = tf.math.reduce_sum(tf.math.abs(image_1 - image_2))
  ssim = tf.image.ssim(image_1, image_2, 2.0)
  psnr = tf.image.psnr(image_1, image_2, 2.0)
  return [L1_distance.numpy(), ssim.numpy()[0], psnr.numpy()[0]]

In [ ]:
def splitMaskMetrics(target, prediction, locations):
    locations = np.squeeze(locations.numpy(), axis = 0)
    ssim_list = []
    psnr_list = []

    for location in locations:
        mask_height = tf.cast(location[0]/2, tf.int32).numpy()
        mask_width = tf.cast(location[1]/2, tf.int32).numpy()
        height_offset = location[2]
        width_offset = location[3]

        targetMask = target[:, height_offset-mask_height:height_offset+mask_height, width_offset-mask_width:width_offset+mask_width,:]
        predictiontMask = prediction[:, height_offset-mask_height:height_offset+mask_height, width_offset-mask_width:width_offset+mask_width, :] 

        ssim = tf.image.ssim(targetMask, predictiontMask, 2.0)
        psnr = tf.image.psnr(targetMask, predictiontMask, 2.0)


        ssim_list.append(ssim.numpy()[0])
        psnr_list.append(psnr.numpy()[0])
    

    l1 = tf.math.reduce_sum(tf.math.abs(target-prediction))

    return [l1.numpy(), sum(ssim_list)/len(ssim_list), sum(psnr_list)/len(psnr_list)]

def get_dataset_metrics(ds):
  #input_image is the inpainted image, GT is the groundtruth
  names = ["L1","SSIM","PSNR"]
  metrics_list = []
  img_path_list = []
  mask_list = []
  
  for (masked_image, image, mask, image_file, locations) in ds:
    prediction = generator(masked_image, training=True) #estimated values for each of 35 classes
    prediction = prediction * mask + image*(1-mask) #insert generated label patches to inverted ground truth

    metrics_list.append(splitMaskMetrics(image, prediction, locations))
    #img_path_list.append(image_file[i].numpy()[0].decode("utf-8"))
    #mask_list.append(mask[i])
    
  
  metrics_df = pd.DataFrame(metrics_list)
  metrics_df.columns = names
  #metrics_df["Img Path"] = img_path_list
  #metrics_df["Mask"] = mask_list
  return metrics_df

In [ ]:
metrics_test = get_dataset_metrics(tf_val)

In [ ]:
with open("/content/drive/MyDrive/Scriptie/json/GAN_metrics.json", 'w') as j:
  json.dump(GAN_metrics, j, indent = 6)

In [ ]:
def reconstruct_img(image_file, mask):
  image = load(image_file, 3)
  image = resize(image, 256, 512)
  masked_image = (1-mask)*image
  image = normalize(image)
  masked_image = normalize(masked_image)
  

  return image, masked_image, mask

In [ ]:
def show_img(img_tuple):
  target, input_image, mask = img_tuple
  target = tf.expand_dims(target, 0)

  predicted = generator(input_image, training=True)
  predicted = predicted * mask + target*(1-mask)
  

  images = [input_image[0], target[0], predicted[0]]
  title = ["Input Image", "Ground Truth", "Predicted Image"]
  plt.figure(figsize=(15, 15))

  for i, image in enumerate(images):
    plt.subplot(3,1,i+1)
    plt.title(title[i])
    plt.imshow(image*0.5+0.5)
    plt.axis('off')
  plt.show()

In [ ]:
rand_idx = randint(0,tf_val.cardinality())
img_idx = rand_idx
RecImages = reconstruct_img(metrics_test["Img Path"][img_idx], metrics_test["Mask"][img_idx])
show_img(RecImages)
print(metrics_test[["L1","SSIM","PSNR"]].loc[[img_idx]])

In [ ]:
def analyse_checkpoints(path_to_ckpts):
  list_ckpts = os.listdir(path_to_ckpts)
  #list_ckpts = list(filter(lambda name: name[-20:] == ".data-00000-of-00001", list_ckpts))
  list_ckpts = [name[:-20] for name in list_ckpts if name[-20:]== ".data-00000-of-00001"]
  metrics = {}


  for ckpt in list_ckpts:
    metrics[ckpt] = {}
    checkpoint.restore(os.path.join(path_to_ckpts, ckpt))
    metrics_df = get_dataset_metrics(tf_val)
    metrics[ckpt]["mean"] = list(metrics_df[["L1","SSIM","PSNR"]].mean())
    metrics[ckpt]["max"] = list(metrics_df[["L1","SSIM","PSNR"]].max())
    metrics[ckpt]["min"] = list(metrics_df[["L1","SSIM","PSNR"]].min())
    metrics[ckpt]["median"] = list(metrics_df[["L1","SSIM","PSNR"]].median())
  
  return metrics

In [ ]:
metrics_ckpts = analyse_checkpoints("/content/drive/MyDrive/Scriptie/ckpt/P2P/upsample")

In [ ]:
for m in ["mean", "median", "min", "max"]:
  print(m)
  for key, value in metrics_ckpts.items():
    print(key, value[m])
  print(" ")

In [ ]:
for i in tf_val_filtered.take(1):
  masked_image, image, mask, image_file = i

#Fine tuning

In [ ]:
# Fill pandas dataframe with the paths to the files
def folder_to_pd(image_dir, semantics_dir):
  image_paths = {"test": [], "train": [], "val": []}
  semantics_paths = {"test": [], "train": [], "val": []}
  label_paths = {"test": [], "train": [], "val": []}


  for data_type in ["test", "train", "val"]:
    image_subdir = os.path.join(image_dir, data_type)
    for root, subdir, files in os.walk(image_subdir):
      subdir.sort()
      files.sort()
      if files:
        for file in files:
          image_paths[data_type].append(os.path.join(root, file))

    semantics_subdir = os.path.join(semantics_dir, data_type)
    for root, subdir, files in os.walk(semantics_subdir):
      subdir.sort()
      files.sort()
      if files:
        for i,file in enumerate(files):
          if file[-12:] == "labelIds.png":
            semantics_paths[data_type].append(os.path.join(root, file))
          if file[-5:] == ".json":
            label_paths[data_type].append(os.path.join(root, file))
  
  test_ds = pd.DataFrame(list(zip(image_paths["test"], semantics_paths["test"], label_paths["test"])), columns =['image', 'labelIds', 'json'])
  train_ds = pd.DataFrame(list(zip(image_paths["train"], semantics_paths["train"], label_paths["train"])), columns =['image', 'labelIds', 'json'])
  val_ds = pd.DataFrame(list(zip(image_paths["val"], semantics_paths["val"], label_paths["val"])), columns =['image', 'labelIds', 'json'])

  return test_ds, train_ds, val_ds

#Function that determines whether there is a person present in the image
def flag_person(json_path):
  person_present = False #Flag to determine whether person(s) are present in image
  with open(json_path) as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()
    for element in jsonObject["objects"]:
      if element["label"] == "person":
        return True
  return False

#function that filters out all the images without images in them
# if filter persons is true, get all images WITHOUT person. If False get all images WITH persons
def filter_ds(ds, filter_person = True):
  json_list = ds["json"]
  drop_rows = [i for i,element in enumerate(json_list) if flag_person(element) == filter_person]
  return ds.drop(drop_rows)

# convert pandas dataframe to tensorflow dataset
def pd_to_ds(pd_df):
  return tf.data.Dataset.from_tensor_slices((pd_df["image"], pd_df["labelIds"], pd_df["json"])) #returns dataset with image and labels for each element in the dataset

In [ ]:
def personFilter(df):
  df = df.reset_index()  # make sure indexes pair with number of rows
  threshold = 35000 
  drop_rows = [] # if less labeled pixels than threshold, drop the row

  for index, row in df.iterrows():
    labels = load(row["labelIds"], 0)
    binary_mask =  tf.math.equal(labels, 24)
    count_labels = tf.math.count_nonzero(binary_mask)
    if count_labels < threshold:
      drop_rows.append(index)
  
  return df.drop(drop_rows)

In [ ]:
def create_person_masks(image_path, labels_path, json_path):
  personID = 24
  riderID = 25  
  
  image = load(image_path, 3)
  image = resize(image, 256, 512)

  labels = load(labels_path, 1)
  labels = resize(labels, 256,512)


  mask_person = tf.math.equal(labels, personID)
  mask_person = tf.cast(mask_person, tf.float32) #zeros where persons are

  mask_rider = tf.math.equal(labels, riderID)
  mask_rider = tf.cast(mask_rider, tf.float32)

  # merge masks
  mask = mask_person + mask_rider

  masked_image = image * (1-mask)
  masked_image = normalize(masked_image)
  image = normalize(image)

  return masked_image, image, mask

In [ ]:
#Create fine tuning datasets
FT_test_ds, FT_train_ds, FT_val_ds = folder_to_pd("/content/cityScapes/img/leftImg8bit", "/content/cityScapes/annotations/gtFine")

In [ ]:
large_persons_train = personFilter(FT_train_ds)
large_persons_val = personFilter(FT_val_ds)

In [ ]:
# Dataset is heeft nu pad naar foto, semantics en json
# grote hoeveelheden labels van mensen
# hiervan masks maken en deze opslaan op drive

In [ ]:
large_persons_train = pd_to_ds(large_persons_train)
large_persons_val = pd_to_ds(large_persons_val)

In [ ]:
person_mask_train = large_persons_train.map(create_person_masks, num_parallel_calls=tf.data.AUTOTUNE)
person_mask_train = person_mask_train.batch(BATCH_SIZE)
person_mask_train = person_mask_train.prefetch(tf.data.AUTOTUNE)

person_mask_val = large_persons_val.map(create_person_masks, num_parallel_calls=tf.data.AUTOTUNE)
person_mask_val = person_mask_val.batch(BATCH_SIZE)
person_mask_val = person_mask_val.prefetch(tf.data.AUTOTUNE)

In [ ]:
def save_mask_ds(ds, path):
  for i, mask in ds.enumerate():
    mask = mask[0]
    name = path + "_" + str(i.numpy())+ ".png"
    tf.keras.utils.save_img(name, mask, data_format= "channels_last")
    
    mirror_name = path + "_" + str(i.numpy()) + "mirror" + ".png"
    reverse = tf.reverse(mask, [1])
    tf.keras.utils.save_img(mirror_name, reverse, data_format= "channels_last")

In [ ]:
#save_mask_ds(person_mask_val, "/content/drive/MyDrive/Scriptie/masks/val/val")

In [ ]:
# dataset with where each element is a tuple (path_to_image, path_to_labels) 
train_noPersons = pd_to_ds(filter_ds(FT_train_ds))
val_noPersons = pd_to_ds(filter_ds(FT_val_ds))

train_persons = pd_to_ds(filter_ds(FT_train_ds, False))
val_persons = pd_to_ds(filter_ds(FT_val_ds, False))

print("Training dataset WITHOUT persons has", train_noPersons.cardinality().numpy(), "elements.")
print("Validation dataset WITHOUT persons has", val_noPersons.cardinality().numpy(), "elements.")
print("")
print("Training dataset WITH persons has", train_persons.cardinality().numpy(), "elements.")
print("Validation dataset WITH persons has", val_persons.cardinality().numpy(), "elements.")

In [ ]:
val_noPersons = val_noPersons.map(create_person_masks, num_parallel_calls=tf.data.AUTOTUNE)
val_noPersons = val_noPersons.batch(BATCH_SIZE)
val_noPersons = val_noPersons.prefetch(tf.data.AUTOTUNE)

In [ ]:
for i, (masked_image, image, mask) in person_mask_val.enumerate():
  prediction = generator(image, training = True)
  prediction = mask * prediction + image * (1-mask)
  fix, axs = plt.subplots(1,3, figsize = (20,20))
  axs[0].imshow(image[0]*0.5+0.5)
  axs[1].imshow(masked_image[0]*0.5+0.5)
  axs[2].imshow(prediction[0]*0.5+0.5)
  axs[0].axis("off")
  axs[1].axis("off")
  axs[2].axis("off")
  plt.show()

  if i ==10:
    break

In [ ]:
for image, mask in val_noPersons.take(1):
  image = image[0]
  plt.imshow(image*0.5+0.5)

In [ ]:
for i, (masked_image, image, mask) in person_mask_val.enumerate():

In [ ]:
plt.imshow(mask[0,:,:,-1])

In [ ]:
masks = tf.data.Dataset.list_files("/content/drive/MyDrive/Scriptie/masks/train/*")
train_noPersons = tf.data.Dataset.zip((train_noPersons, masks))

In [ ]:
masks_val = tf.data.Dataset.list_files("/content/drive/MyDrive/Scriptie/masks/val/*")
val_noPersons = tf.data.Dataset.zip((val_noPersons, masks_val))

In [ ]:
def person_mask(labelIm):
  #labelID used for persons is 24 and for rider is 25
  personID = 24
  riderID = 25
  mask_person = tf.math.equal(labelIm, personID)
  mask_person = tf.cast(mask_person, tf.float32) #zeros where persons are

  mask_rider = tf.math.equal(labelIm, riderID)
  mask_rider = tf.cast(mask_rider, tf.float32)

  # merge masks
  mask = mask_person + mask_rider

  return mask

In [ ]:
def load_images_CS(info, mask):
  mask = load(mask, 0)
  mask = mask /255

  image_file, labels_file, json_file = info
  image = load(image_file, 3)
  image = resize(image, 256,512)
  masked_image = image * (1-mask)
  image = normalize(image)
  masked_image = normalize(masked_image)
  
  

  labels = load(labels_file, 0)
  labels = resize(labels, 256,512)
  masked_labels = labels * (1-mask)

  return image, masked_image, labels, masked_labels, mask

In [ ]:
"/content/drive/MyDrive/Scriptie/Cityscapes/No_persons"

In [ ]:
val_noPersons

In [ ]:
finetune_train = train_noPersons.shuffle(train_noPersons.cardinality(), reshuffle_each_iteration=False)
finetune_train = finetune_train.map(load_images_CS, num_parallel_calls=tf.data.AUTOTUNE)
finetune_train = finetune_train.batch(BATCH_SIZE)
#train_noPersons = train_noPersons.cache("/content/temporary.tfcache")
finetune_train = finetune_train.prefetch(tf.data.AUTOTUNE)

finetune_val = val_noPersons.shuffle(val_noPersons.cardinality(), reshuffle_each_iteration=False)
finetune_val = finetune_val.map(load_images_CS, num_parallel_calls=tf.data.AUTOTUNE)
finetune_val = finetune_val.batch(BATCH_SIZE)
#train_noPersons = train_noPersons.cache("/content/temporary.tfcache")
finetune_val = finetune_val.prefetch(tf.data.AUTOTUNE)

In [ ]:
for image, masked_image, labels, masked_labels, mask in finetune_train.take(1):
  plt.imshow(mask[0,:,:,-1])

In [ ]:
plt.imshow(masked_image[0]*0.5+0.5)

In [ ]:
prediction = generator(masked_image, training = True)
prediction = mask * prediction + image * (1-mask)

In [ ]:
plt.imshow(prediction[0]*0.5+0.5)

In [ ]:
plt.imshow(image[0]*0.5+0.5)

## Train

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-5, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-5, beta_1=0.5)

In [ ]:
@tf.function
def train_epoch(input_image, mask, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True) #discriminator takes predicted labels

    gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                              discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))
  
  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss
    
  
def getValAccLoss(input_image, mask, target): 
  gen_output = generator(input_image, training=True)
  gen_output = gen_output*mask+ target*(1-mask) #convert the 35 channel output of the generator to a 1 channel output with labels

  disc_real_output = discriminator([input_image, target], training=True)
  disc_generated_output = discriminator([input_image, gen_output], training=True) #discriminator takes predicted labels

  gen_total_loss, gen_gan_loss, gen_CE_loss = generator_loss(disc_generated_output, gen_output, target) #CE loss of 35 channel output
  disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  return gen_total_loss, gen_gan_loss, gen_CE_loss, disc_loss

In [ ]:
log_dir= '/content/drive/MyDrive/Scriptie/logs/im_generator/P2P/TransposeConv/epoch/finetune/'
summary_writer_train = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_TrainFT")
summary_writer_val = tf.summary.create_file_writer(log_dir + "fit/combined/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_ValFT")

In [ ]:
def fit(train_ds, test_ds, epochs):
  #image, masked_image, labels, masked_labels, mask
  example_target, example_input, labels_test, masked_labels_test, mask_test = next(iter(test_ds.take(1)))
  example_input, example_target, mask_test
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    generate_images(generator, example_input, example_target, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(target, input_image, labels, masked_labels, mask) in train_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epoch(input_image, mask, target)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)

    
    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(target, input_image, labels, masked_labels, mask) in test_ds.enumerate():
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLoss(input_image, mask, target)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 5 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)


In [ ]:
checkpoint_prefix = "/content/drive/MyDrive/Scriptie/ckpt/finetune/GAN/ckpt"

In [ ]:
checkpoint.save(file_prefix=checkpoint_prefix)

In [ ]:
fit(finetune_train, finetune_val, 10)

In [ ]:
dataset = tf_train_filtered.take(9)
for images in tf_train_filtered.take(9):
  images = images
  plt.figure(figsize=(15, 15))
  num_images = 9
  
  for i in range(num_images):
    plt.subplot(3, 3, i+1)
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(images[0][i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
title = ["masked image", "GT", "mask", "prediction"]
predicted = generator(masked_image)
predicted = predicted*mask+image*(1-mask)
display_list = masked_image[0], image[0], mask[0,:,:,0], predicted[0]
plt.figure(figsize=(30, 30))
for i in range(4):
  plt.subplot(1, 4, i+1)
  plt.title(title[i])
  # Getting the pixel values in the [0, 1] range to plot.
  plt.imshow(display_list[i] * 0.5 + 0.5)
  plt.axis('off')
plt.show()

# Reducing data

In [ ]:
# loop
## reduce training data
## reset model
## train model on data
## get metrics (L1, PSNR, SSIM) means
## save metrics

In [ ]:
"""""
std_dict = {'1.0': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.9': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.8': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.7': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.6': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.5': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.4': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.3': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.2': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.1': {'L1': [], 'PSNR': [], 'SSIM': []}}
"""""

In [ ]:
"""""
metrics_dict = {'1.0': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.9': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.8': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.7': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.6': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.5': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.4': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.3': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.2': {'L1': [], 'PSNR': [], 'SSIM': []},
 '0.1': {'L1': [], 'PSNR': [], 'SSIM': []}}
"""""

In [ ]:
with open('/content/drive/MyDrive/Scriptie/json/metrics_30000_steps_v2.json') as json_file:
    metrics_dict = json.load(json_file)

with open('/content/drive/MyDrive/Scriptie/json/std_30000_steps.json') as json_file:
    std_dict = json.load(json_file)

In [ ]:
def drop_rows(df, n_rows):
  drop_rows = np.random.choice(df.index, n_rows, replace=False)
  return df.drop(drop_rows)

def tf_dataset(train_ds, val_ds):
  #prepare training dataset
  tf_train = train_ds.shuffle(train_ds.cardinality(), reshuffle_each_iteration=True)
  tf_train = tf_train.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
  tf_train = tf_train.batch(BATCH_SIZE)
  tf_train = tf_train.prefetch(tf.data.AUTOTUNE)
  
  #prepare validation dataset
  tf_val = val_ds.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
  tf_val = tf_val.batch(BATCH_SIZE)
  tf_val = tf_val.prefetch(tf.data.AUTOTUNE)

  return tf_train, tf_val

def reduce_data(train_df, val_df, reduce_percentage):
  print(f"Start reducing the dataset by {reduce_percentage*100}%")
  #determine number of rows to drop
  n_drop_train = round(train_df.shape[0]*(1-reduce_percentage))
  
  #randomly drop rows
  train_subset = drop_rows(train_df, n_drop_train)
  #val_subset = drop_rows(val_df, n_drop_val)
  print(f"Dropped {n_drop_train} rows from the training set, training dataset now contains {train_subset.shape[0]} images")
  #print(f"Dropped {n_drop_val} rows from the validation set")

  #make tensorflow datasets from the pandas dataframes
  train_ds = tf.data.Dataset.from_tensor_slices(train_subset["Image"])
  val_ds = tf.data.Dataset.from_tensor_slices(val_df["Image"])
  train_ds, val_ds = tf_dataset(train_ds, val_ds)
  print(f"Tensorflow training dataset: {train_ds.cardinality()}. Tensorflow validation dataset: {val_ds.cardinality()}")
  
  return train_ds, val_ds

In [ ]:
generator = Generator()
discriminator = Discriminator()
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
@tf.function
def train_step(input_image, mask, target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator([input_image], training=True) #generate image based on input
    gen_output = gen_output*mask+ target*(1-mask) #combine generated patches with valid pixels from ground truth

    disc_real_output = discriminator([input_image, target], training=True) #calculate discriminator loss for ground truth images
    disc_generated_output = discriminator([input_image, gen_output], training=True) #calculate discriminator loss for predicted images

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))

In [ ]:
proportion = 1.0

In [ ]:
train_ds_subset, val_ds_subset = reduce_data(train_pd, val_pd, proportion)

In [ ]:
#train the model
fit(train_ds_subset, val_ds_subset, 30000)

print("Gathering metrics")
metrics_train = get_dataset_metrics(train_ds_subset)
metrics_val = get_dataset_metrics(val_ds_subset)

for key in metrics_dict[str(proportion)].keys():
  metrics_dict[str(proportion)][key].append([str(metrics_train[key].mean()), str(metrics_val[key].mean())])
  std_dict[str(proportion)][key].append([str(metrics_train[key].std()), str(metrics_val[key].std())])

with open('/content/drive/MyDrive/Scriptie/json/metrics_30000_steps_v2.json', 'w') as j:
  json.dump(metrics_dict, j, indent = 6)

with open('/content/drive/MyDrive/Scriptie/json/std_30000_steps.json', 'w') as j:
  json.dump(std_dict, j, indent = 6)

display.clear_output(wait=True)
print(len(metrics_dict[str(proportion)]["L1"]))

# Plot

In [ ]:
def convert_dict(data):
  # Create dictionaries for training and validation scores
  train_dict = {}
  val_dict = {}

  for k, v in data.items():
      for metric, scores in v.items():
          if metric not in train_dict:
              train_dict[metric] = []
          if metric not in val_dict:
              val_dict[metric] = []
          train_score = sum([float(x[0]) for x in scores]) / len(scores)
          val_score = sum([float(x[1]) for x in scores]) / len(scores)
          if metric == "L1":
            train_score = round(train_score)
            val_score = round(val_score)
          elif metric == "PSNR":
            train_score = round(train_score, 2)
            val_score = round(val_score, 2)
          elif metric == "SSIM":
            train_score = round(train_score, 4)
            val_score = round(val_score, 4)
          
          train_dict[metric].append(train_score)
          val_dict[metric].append(val_score)
  
  return train_dict, val_dict

In [ ]:
train_dict, val_dict = convert_dict(metrics_dict)
train_dict_std, val_dict_std = convert_dict(std_dict)

In [ ]:
l1_val

In [ ]:
psnr_val

In [ ]:
ssim_val

In [ ]:
std_l1_v

In [ ]:
np.linspace(1.0, 0.1, 10)

In [ ]:
l1_train = train_dict["L1"]
l1_val = val_dict["L1"]
psnr_train = train_dict["PSNR"]
psnr_val = val_dict["PSNR"]
ssim_train = train_dict["SSIM"]
ssim_val = val_dict["SSIM"]

std_l1_t = train_dict_std["L1"]
std_l1_v = val_dict_std["L1"]
std_psnr_t = train_dict_std["PSNR"]
std_psnr_v = val_dict_std["PSNR"]
std_ssim_t = train_dict_std["SSIM"]
std_ssim_v = val_dict_std["SSIM"]

x = np.linspace(1.0, 0.1, 10)

# Create subplots
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

# Plot data with error bars
axs[0].errorbar(x, l1_train, yerr=std_l1_t, label='Train', solid_capstyle='projecting', capsize=5)
axs[0].errorbar(x, l1_val, yerr=std_l1_v, label='Validation', solid_capstyle='projecting', capsize=5)

axs[1].errorbar(x, psnr_train, yerr=std_psnr_t, label='Train', solid_capstyle='projecting', capsize=5)
axs[1].errorbar(x, psnr_val, yerr=std_psnr_v, label='Validation', solid_capstyle='projecting', capsize=5)

axs[2].errorbar(x, ssim_train, yerr=std_ssim_t, label='Train', solid_capstyle='projecting', capsize=5)
axs[2].errorbar(x, ssim_val, yerr=std_ssim_v, label='Validation', solid_capstyle='projecting', capsize=5)

# Set plot properties
for i, ax in enumerate(axs):
    ax.set_xlabel('Proportion of Training data')
    ax.set_xticks(x)
    if i == 0:
      ax.set_ylabel('L1')
    elif i == 1:
      ax.set_ylabel('PSNR')
    elif i == 2:
      ax.set_ylabel('SSIM')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Show plot
plt.show()

In [ ]:
import matplotlib.image as mpimg

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(16, 12))
input = "/content/drive/MyDrive/Scriptie/Test_imgs/0.1/berlin/input.png"
gt = "/content/drive/MyDrive/Scriptie/Test_imgs/0.1/berlin/target.png"
path = "/content/drive/MyDrive/Scriptie/Test_imgs/"
im_path = "/berlin/prediction.png"
images = np.array([input, gt] + [path+str((x+1)/10)+im_path for x in range(10)]).reshape(4,3)

titles = np.array(["Input", "Ground truth"] + [str((x+1)/10) for x in range(10)]).reshape(4,3)

for i, row in enumerate(axs):
  for j, im in enumerate(row):
    img = mpimg.imread(images[i][j])
    im.imshow(img)
    im.axis("off")
    im.set_title(titles[i][j])
plt.show()

In [ ]:
path = "/content/cityScapes/img/leftImg8bit/test"
test_fotos = ["/berlin/berlin_000004_000019_leftImg8bit.png",
              "/bielefeld/bielefeld_000000_020900_leftImg8bit.png",
              "/bonn/bonn_000018_000019_leftImg8bit.png",
              "/leverkusen/leverkusen_000045_000019_leftImg8bit.png",
              "/mainz/mainz_000000_010417_leftImg8bit.png",
              "/munich/munich_000242_000019_leftImg8bit.png"]

target_path = "/content/drive/MyDrive/Scriptie/Test_imgs" + "/" + str(proportion)
cities = ["berlin", "bielefeld", "bonn", "leverkusen", "mainz", "munich"]
names = ["input", "target", "mask", "prediction"]


for idx, test_foto in enumerate(test_fotos):
  path_to_im = path + test_foto
  _, target, _, _, _ = load_images(path_to_im)
  mask_path = os.path.join("/content/drive/MyDrive/Scriptie/Test_imgs/0.1", cities[idx]) + "/mask.png"
  mask = load(mask_path, 1)/255
  inp = tf.where(tf.cast(mask, tf.bool), -1, target)
  gen_output = generator([tf.expand_dims(inp, 0)], training=True)
  gen_output = gen_output*mask+ target*(1-mask)
  gen_output = np.squeeze(gen_output, 0)
  ims = [inp, target, mask, gen_output]

  for i in range(6):
    for j in range(4):
      target_str = os.path.join(target_path, cities[i], names[j]) + ".png"
      tf.keras.utils.save_img(target_str, ims[j])

In [ ]:
for elements in train_ds_subset.take(1):
  (masked_image, image, mask, image_file, locations) = elements

In [ ]:
small_batch = val_ds_subset.take(1)

In [ ]:
get_dataset_metrics(small_batch)

In [ ]:
for masked_image, image, mask, image_file, locations in small_batch:
  prediction = generator(masked_image, training=True) #estimated values for each of 35 classes
  prediction = prediction * mask + image*(1-mask)

  for i in range(prediction.shape[0]):
    print(splitMaskMetrics_single(image[i], prediction[i], locations[i]))
    fix,axs = plt.subplots(1,3, figsize=(15,15))
    axs[0].imshow(masked_image[i]*0.5+0.5)
    axs[1].imshow(image[i]*0.5+0.5)
    axs[2].imshow(prediction[i]*0.5+0.5)
    plt.axis("off")
    plt.show()

In [ ]:
metrics_train = get_dataset_metrics(train_ds_subset)
metrics_val = get_dataset_metrics(val_ds_subset)

In [ ]:
print(metrics_train.mean())
print(metrics_val.mean())

In [ ]:
print(list(metrics_dict.keys()))

In [ ]:
def convert_dtype(dtype, l1, l2):
  return list(map(dtype, l1)), list(map(dtype, l2))

In [ ]:
l1_t, l1_v = list(zip(*metrics_dict["L1"]))
psnr_t, psnr_v = list(zip(*metrics_dict["PSNR"]))
ssim_t, ssim_v = list(zip(*metrics_dict["SSIM"]))

In [ ]:
l1_t, l1_v = convert_dtype(float, list(l1_t), list(l1_v))
psnr_t, psnr_v = convert_dtype(float, list(psnr_t), list(psnr_v))
ssim_t, ssim_v = convert_dtype(float, list(ssim_t), list(ssim_v))

In [ ]:
def mean(data):
  return sum(data)/len(data)

In [ ]:
test = ["1.0", "2.0"]
for i in test:
  print(type(i), i)
  i = float(i)
  print(type(i), i)

In [ ]:
def DataDict(dictionary, key):
  train_list = []
  val_list = []
  x = list(metrics_dict.keys())
  for i in x:
    values = dictionary[i][key]
    train_data, val_data = list(zip(*values))
    train_data, val_data = list(map(float, train_data)), list(map(float, val_data))
    train_list.append(mean(train_data))
    val_list.append(mean(val_data))
    
  
  return train_list, val_list

In [ ]:
def plot_data(train, val, title, x_axis, y_axis):
  x = np.linspace(1.0, 0.1, 10, endpoint=True)
  # plot lines
  plt.plot(x, train, label = "training data")
  plt.plot(x, val, label = "validation data")
  plt.title(title)
  plt.xlabel(x_axis)
  plt.ylabel(y_axis)
  plt.xticks(x)
  plt.legend()
  plt.show()

In [ ]:
title = "L1 data decrease curve"
x_axis = "Proportion of trainingset used"
y_axis = "L1"
l1_t, l1_v = DataDict(metrics_dict, "L1")
plot_data(l1_t, l1_v, title, x_axis, y_axis)

In [ ]:
title = "PSNR data decrease curve"
x_axis = "Proportion of trainingset used"
y_axis = "PSNR"
psnr_t, psnr_v = DataDict(metrics_dict, "PSNR")
plot_data(psnr_t, psnr_v, title, x_axis, y_axis)

In [ ]:
title = "SSIM data decrease curve"
x_axis = "Proportion of trainingset used"
y_axis = "SSIM"
ssim_t, ssim_v = DataDict(metrics_dict, "SSIM")
plot_data(ssim_t, ssim_v, title, x_axis, y_axis)

In [ ]:
for i in tf_val.take(1):
  masked_image, image, mask, image_file, locations = i
plt.imshow(masked_image[0]*0.5+0.5)

In [ ]:
generate_images(generator, masked_image, image, mask)
prediction = generator([masked_image], True)
prediction = mask*prediction + (1-mask)*image
data = splitMaskMetrics(image, prediction, locations)
print(data)

In [ ]:
data = splitMaskMetrics(image, prediction, locations)
print(data)

In [ ]:
print(tf.image.ssim(image, prediction, 255))
print(tf.image.psnr(image, prediction, 255))
print(tf.math.reduce_sum(tf.math.abs(image - prediction)))

In [ ]:
def fitSEMGAN(train_ds, test_ds, epochs):
  masked_image_test, image_test, semantics_masked_test, semantics_test, mask_test, _, _ = next(iter(test_ds.take(1)))
  start = time.time()
  total_ims = tf.cast(train_ds.cardinality(), tf.float32)
  total_ims_val = tf.cast(test_ds.cardinality(), tf.float32)

  for epoch in range(epochs):
    metrics_dict = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    
    #semantics used for the example
    semantics_predicted_test = gen_to_label(generator_semantics(semantics_masked_test, training=True)) # turn the generator output into labels
    semantics_predicted_test = semantics_predicted_test*tf.cast(mask_test, tf.int32)+semantics_test*(1-tf.cast(mask_test, tf.int32)) # combine the prediction with the ground truth
    semantics_sparse_test = label_to_sparse(semantics_predicted_test) # make a sparse matrix as input to the generator
    generate_images(generatorSEMGAN, masked_image_test, image_test, semantics_predicted_test, semantics_sparse_test, mask_test)

    print("Start training of epoch: ", epoch+1)
    for step,(masked_image, image, semantics_masked, semantics, mask, _, _) in train_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True)) # turn the generator output into labels
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32)) # combine the prediction with the ground truth
      semantics_predicted = label_to_sparse(semantics_predicted) # make a sparse matrix as input to the generator
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = train_epochSEMGAN(masked_image, image, semantics_predicted, mask)
      metrics_list = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
      
      for i,(key,value) in enumerate(metrics_dict.items()):
        metrics_dict[key] = value + metrics_list[i]
      
      if (step+1) % 50 == 0:
        print('.', end='', flush=True)

    with summary_writer_train.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict["gen_total_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict["gen_gan_loss"]/total_ims, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict["gen_L1_loss"]/total_ims, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict["disc_loss"]/total_ims, step=epoch)

    fas
    # Gather the metrics for the validation set
    metrics_dict_val = {"gen_total_loss":0, "gen_gan_loss":0, "gen_L1_loss":0, "disc_loss":0}
    print("Gathering validation set metrics...")
    for step,(masked_image, image, semantics_masked, semantics, mask, _, _) in test_ds.enumerate():
      semantics_predicted = gen_to_label(generator_semantics(semantics_masked, training=True))
      semantics_predicted = semantics_predicted * tf.cast(mask, tf.int32) + semantics *(1-tf.cast(mask, tf.int32))
      semantics_predicted = label_to_sparse(semantics_predicted)
      gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss = getValAccLossSEMGAN(masked_image, image, semantics_predicted, mask)
      metrics_list_val = (gen_total_loss, gen_gan_loss, gen_L1_loss, disc_loss)
        
      for i,(key,value) in enumerate(metrics_dict_val.items()):
        metrics_dict_val[key] = value + metrics_list_val[i]
    
    with summary_writer_val.as_default():
      tf.summary.scalar('gen_total_loss', metrics_dict_val["gen_total_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_gan_loss', metrics_dict_val["gen_gan_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('gen_L1_loss', metrics_dict_val["gen_L1_loss"]/total_ims_val, step=epoch)
      tf.summary.scalar('disc_loss', metrics_dict_val["disc_loss"]/total_ims_val, step=epoch)

    display.clear_output(wait=True)
    print(f'Time taken for 1 epoch: {time.time()-start:.2f} sec\n')
    start = time.time()

    # Save (checkpoint) the model every 5 epochs
    #if epoch % 10 == 0 & epoch !=0: 
    #  checkpoint.save(file_prefix=checkpoint_prefix)

# Plot

In [ ]:
checkpoint.restore("/content/drive/MyDrive/Scriptie/ckpt/P2P/TransposeConv/ckpt-1")

In [ ]:
example_input, example_target, mask_test, path, locations = next(iter(tf_val.take(1)))

In [ ]:
plt.imshow(example_target[0]*0.5+0.5)

In [ ]:
prediction = generator(example_input, True)
prediction = mask_test * prediction + example_target * (1-mask_test)
plt.imshow(prediction[0]*0.5+0.5)

In [ ]:
splitMaskMetrics(example_target, prediction, locations)

In [ ]:
metrics(example_target, prediction)

In [ ]:
import os
import glob

In [ ]:
for root, dirs, files in os.walk("/content/drive/MyDrive/Scriptie/Test_imgs", topdown = True):
   for name in files:
      print(os.path.join(root, name))